In [ ]:
using Pkg
pkg"activate ."

## Usage

Run the `./run_lcm_timing.sh` script, which will simulate the robot for 10 seconds and generate an LCM log.

In [ ]:
using Plots
using BotCoreLCMTypes
using LCMCore

In [ ]:
controller_dts = let
    lcmlog = LCMLog("lcmlog-2018-09-15.00")  # Change this to your actual LCM log filename
    last_state_timestamp = 0
    last_state_utime = 0
    controller_dts = Int64[]

    while isgood(lcmlog)
        event = LCMCore.read_next_event(lcmlog)
        if event === nothing
            break
        end
        channel = unsafe_string(event.channel, event.channellen)
        if channel == "EST_ROBOT_STATE"
            msgdata = unsafe_wrap(Vector{UInt8}, event.data, event.datalen)
            msg = decode(msgdata, robot_state_t)
#             println("state:\t\t", msg.utime)
            last_state_timestamp = event.timestamp
            last_state_utime = msg.utime
        elseif channel == "ATLAS_COMMAND"
            msgdata = unsafe_wrap(Vector{UInt8}, event.data, event.datalen)
            msg = decode(msgdata, atlas_command_t)
#             println("control:\t", msg.utime)
            @assert isapprox(msg.utime, last_state_utime, atol=1)
            push!(controller_dts, event.timestamp - last_state_timestamp)
        end
    end
    controller_dts
end

close(lcmlog)

In [ ]:
plt = histogram(controller_dts[30:end] ./ 1000, legend=false)
xlabel!(plt, "Controller dt (ms)")
ylabel!(plt, "Frequency")
savefig("controller_dt_histogram.svg")
savefig("controller_dt_histogram.png")
plt